In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import rasterio
import rioxarray
import numpy as np
import matplotlib.pyplot as plt
import gdown
import geopandas as gpd

In [ ]:
#Download KID-13 LiDAR DEMs
url = 'https://drive.google.com/drive/u/1/folders/1HZYqsozcfNUbnPZT1gU07FXrz-1QCaHJ'
k13_lidar_wsyr1='../data/dems/kid_lidar_wsyr1'
gdown.download_folder(url, output=k13_lidar_wsyr1, quiet=False, use_cookies=False)

url = 'https://drive.google.com/drive/u/1/folders/1E_m0zWlg11__oyjS6zkb0w1M6u3-Eww5'
k13_lidar_dsyr1='../data/dems/kid_lidar_dsyr1'
gdown.download_folder(url, output=k13_lidar_dsyr1, quiet=False, use_cookies=False)

url = 'https://drive.google.com/drive/u/1/folders/11EnAN2Z3fBMALL2ZcgCOhuwTz06w47Dm'
k13_lidar_wsyr2='../data/dems/kid_lidar_wsyr2'
gdown.download_folder(url, output=k13_lidar_wsyr2, quiet=False, use_cookies=False)

In [ ]:
#Download KID-13 SfM DSMs
url = 'https://drive.google.com/drive/u/1/folders/1AN05fgxtuUhgEAs2aVdNehtKvndVARiz'
sfm_dem_folder='../data/dems/kid_sfm_dems'
gdown.download_folder(url, output=sfm_dem_folder, quiet=False, use_cookies=False)

In [ ]:
url = 'https://drive.google.com/drive/u/1/folders/1L_Qrrjmk-edLfX4yEmO_qDMSZQAIa-8r'
mask_folder='../data/vectors/kid_mask'
gdown.download_folder(url, output=mask_folder, quiet=False, use_cookies=False)

In [ ]:
lidar_dsm_files = sorted(Path(lidar_dem_folder).glob('*.tif'))
lidar_dsm_files = [i.as_posix() for i in lidar_dsm_files]

sfm_dsm_files = sorted(Path(sfm_dem_folder).glob('*.tif'))
sfm_dsm_files = [i.as_posix() for i in sfm_dsm_files]

In [ ]:
lidar_dsm_files, sfm_dsm_files

In [ ]:
mask_path = '../data/vectors/kid_mask/KID13_mask.shp'

# grab shapefile crs to write to rasters
crs = gpd.read_file(mask_path).crs
mask_gdf = gpd.read_file(mask_path)

In [ ]:
lidar = dict()
lidar_clip = dict()
for i in range(len(lidar_dsm_files)):
    lidar[i] = rioxarray.open_rasterio(lidar_dsm_files[i]).squeeze()
    lidar[i].rio.write_crs(crs, inplace=True)
    lidar_clip[i] = lidar[i].rio.clip(mask_gdf.geometry)
    lidar_clip[i].rio.write_nodata(-9999, encoded=True, inplace=True)

lidar_reproj = dict()
for x in range(len(lidar_dsm_files)-1):
    lidar_reproj[x+1] = lidar_clip[x+1].rio.reproject_match(lidar_clip[0], resampling = 3)

sfm = dict()
sfm_clip = dict()
for j in range(len(sfm_dsm_files)):
    sfm[j] = rioxarray.open_rasterio(sfm_dsm_files[j]).squeeze()
    sfm[j].rio.write_crs(crs, inplace=True)
    sfm_clip[j] = sfm[j].rio.clip(mask_gdf.geometry)
    sfm_clip[j].rio.write_nodata(-9999, encoded=True, inplace=True)

sfm_reproj = dict()
for y in range(len(sfm_dsm_files)-1):
    sfm_reproj[y+1] = sfm_clip[y+1].rio.reproject_match(sfm_clip[0], resampling = 3)


In [ ]:
lidar_dsm_files

In [ ]:
#clipping using rioxarray
# Wet season, year 1; LiDAR
K13_20201109_LiDAR_clipped = K13_20201109_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210513_LiDAR_clipped = K13_20210513_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210513_LiDAR_reproj = K13_20210513_LiDAR_clipped.rio.reproject_match(K13_20201109_LiDAR_clipped, resampling=3)
K13_20201109_LiDAR_clipped.rio.to_raster('20201109_KID13_LiDAR.tif')
K13_20210513_LiDAR_reproj.rio.to_raster('20210513_KID13_LiDAR.tif')

# Wet season, year 1; sfm
K13_20201109_sfm_clipped = K13_20201109_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210208_sfm_clipped = K13_20210208_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210406_sfm_clipped = K13_20210406_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210513_sfm_clipped = K13_20210513_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210208_sfm_reproj = K13_20210208_sfm_clipped.rio.reproject_match(K13_20201109_sfm_clipped, resampling=3)
K13_20210406_sfm_reproj = K13_20210406_sfm_clipped.rio.reproject_match(K13_20201109_sfm_clipped, resampling=3)
K13_20210513_sfm_reproj = K13_20210513_sfm_clipped.rio.reproject_match(K13_20201109_sfm_clipped, resampling=3)
K13_20201109_sfm_clipped.rio.to_raster('20201109_KID13_sfm.tif')
K13_20210208_sfm_reproj.rio.to_raster('20210208_KID13_sfm.tif')
K13_20210406_sfm_reproj.rio.to_raster('20210406_KID13_sfm.tif')
K13_20210513_sfm_reproj.rio.to_raster('20210513_KID13_sfm.tif')

# Dry season, year 1; LiDAR
K13_20210604_LiDAR_clipped = K13_20210604_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210913_LiDAR_clipped = K13_20210913_LiDAR_full.rio.clip(mask_gdf.geometry)
K13_20210913_LiDAR_reproj = K13_20210913_LiDAR_clipped.rio.reproject_match(K13_20210604_LiDAR_clipped, resampling=3)
K13_20210604_LiDAR_clipped.rio.to_raster('20210604_KID13_LiDAR.tif')
K13_20210913_LiDAR_reproj.rio.to_raster('20210913_KID13_LiDAR.tif')

# Dry season, year 1; sfm
K13_20210604_sfm_clipped = K13_20210604_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210819_sfm_clipped = K13_20210819_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210913_sfm_clipped = K13_20210913_sfm_full.rio.clip(mask_gdf.geometry)
K13_20210819_sfm_reproj = K13_20210819_sfm_clipped.rio.reproject_match(K13_20210604_sfm_clipped, resampling=3)
K13_20210913_sfm_reproj = K13_20210913_sfm_clipped.rio.reproject_match(K13_20210604_sfm_clipped, resampling=3)
K13_20210604_sfm_clipped.rio.to_raster('20210604_KID13_sfm.tif')
K13_20210819_sfm_reproj.rio.to_raster('20210819_KID13_sfm.tif')
K13_20210913_sfm_reproj.rio.to_raster('20210913_KID13_sfm.tif')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8,4))

m0 = ax[0].imshow(K13_20201109_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[0].set_title('KID-13 2020-11-09\nLiDAR')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210513_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[1].set_title('KID-13 2021-05-13\nLiDAR')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

diff = K13_20210513_LiDAR_reproj-K13_20201109_LiDAR_clipped
m2 = ax[2].imshow(diff, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[2].set_title('KID-13\n2020-11-09 to 2021-05-13\nLiDAR')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8,4))

m0 = ax[0].imshow(K13_20210604_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[0].set_title('KID-13 2021-06-04\nLiDAR')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210513_LiDAR_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[1].set_title('KID-13 2021-09-13\nLiDAR')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

diff = K13_20210913_LiDAR_reproj-K13_20210604_LiDAR_clipped
m2 = ax[2].imshow(diff, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[2].set_title('KID-13\n2021-06-04 to 2021-09-13\nLiDAR')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(15,4))

m0 = ax[0].imshow(K13_20201109_sfm_clipped, cmap = 'viridis', vmin=501, vmax=506)
ax[0].set_title('KID-13 2020-11-09\nSfM')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210208_sfm_reproj, cmap = 'viridis', vmin=501, vmax=506)
ax[1].set_title('KID-13 2021-02-08\nSfM')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

m2 = ax[2].imshow(K13_20210406_sfm_reproj, cmap = 'viridis', vmin=501, vmax=506)
ax[2].set_title('KID-13 2021-04-06\nSfM')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());

m3 = ax[3].imshow(K13_20210513_sfm_reproj, cmap = 'viridis', vmin=501, vmax=506)
ax[3].set_title('KID-13 2021-05-13\nSfM')
fig.colorbar(m3, label='Elevation (m)', extend='both', ax=ax[3])
ax[3].set_facecolor('k')
ax[3].set_xticks(())
ax[3].set_yticks(());

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15,4))

m0 = ax[0].imshow(K13_20210208_sfm_reproj-K13_20201109_sfm_clipped, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[0].set_title('KID-13\n2020-11-09 to 2021-02-08\nSfM')
fig.colorbar(m0, label='Elevation (m)', extend='both', ax=ax[0])
ax[0].set_facecolor('k')
ax[0].set_xticks(())
ax[0].set_yticks(());

m1 = ax[1].imshow(K13_20210406_sfm_reproj-K13_20201109_sfm_clipped, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[1].set_title('KID-13\n2020-11-09 to 2021-04-06\nSfM')
fig.colorbar(m1, label='Elevation (m)', extend='both', ax=ax[1])
ax[1].set_facecolor('k')
ax[1].set_xticks(())
ax[1].set_yticks(());

m2 = ax[2].imshow(K13_20210513_sfm_reproj-K13_20201109_sfm_clipped, cmap = 'RdBu', vmin=-0.05, vmax=0.05)
ax[2].set_title('KID-13\n2020-11-09 to 2021-05-13\nSfM')
fig.colorbar(m2, label='Elevation (m)', extend='both', ax=ax[2])
ax[2].set_facecolor('k')
ax[2].set_xticks(())
ax[2].set_yticks(());